In [1]:
install.packages("datateachr")
library(tidyverse)
library(infer)
library(gridExtra)
library(testthat)
library(digest)
library(broom)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Warning message:
“package ‘datateachr’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine



Attaching package: ‘testthat’


The following objec

In [2]:
stars <- read.csv("6-class.csv")

head(stars)

,Temperature..K.,Luminosity.L.Lo.,Radius.R.Ro.,Absolute.magnitude.Mv.,Star.type,Star.color,Spectral.Class
,<int>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,3068,0.002400,0.1700,16.12,0,Red,M
2,3042,0.000500,0.1542,16.60,0,Red,M
3,2600,0.000300,0.1020,18.70,0,Red,M
4,2800,0.000200,0.1600,16.65,0,Red,M
5,1939,0.000138,0.1030,20.06,0,Red,M
6,2840,0.000650,0.1100,16.98,0,Red,M


In [20]:
stars <- mutate(stars, Star.type = as_factor(Star.type))
dwarfs <- stars %>%
filter(Star.type != 5) %>%
filter(Star.type != 4) %>%
filter(Star.type != 3)

head(dwarfs)

,Temperature..K.,Luminosity.L.Lo.,Radius.R.Ro.,Absolute.magnitude.Mv.,Star.type,Star.color,Spectral.Class
,<int>,<dbl>,<dbl>,<dbl>,<fct>,<chr>,<chr>
1,3068,0.002400,0.1700,16.12,0,Red,M
2,3042,0.000500,0.1542,16.60,0,Red,M
3,2600,0.000300,0.1020,18.70,0,Red,M
4,2800,0.000200,0.1600,16.65,0,Red,M
5,1939,0.000138,0.1030,20.06,0,Red,M
6,2840,0.000650,0.1100,16.98,0,Red,M


# ANOVA

ANOVA (Analysis of Variance) is a test to determine whether the difference between means across three or more groups exist by analyzing the overall and within-group variance. ANOVA test calculates F-value, which is the ratio of the variability between group means and the variation within groups, and corresponding p-value. If F-value is significantly greater than 1, the p-value is going to be small, we conclude that the variability between group means exists. In this project, since our goal is to analyze the difference between temperature and luminosity of three groups of dwarf planets, ANOVA becomes very appropriate and effective. In this section, we will conduct two ANOVA tests across three dwarf groups on both variables, and analyze the results.

### ANOVA for Temperature Across all Three Dwarf Types

We start with temperature. Before actually starting the ANOVA test, we need to do a little bit of cleanup, select the columns that we want, which are Star Type and Temperature.

In [4]:
# filter out the target column Temperature to get the tidy data set
dwarfs_temp <- dwarfs %>%
               select(Temperature..K., Star.type)
head(dwarfs_temp)

,Temperature..K.,Star.type
,<int>,<fct>
1,3068,0
2,3042,0
3,2600,0
4,2800,0
5,1939,0
6,2840,0


In the following two blocks of code, we perform ANOVA test on temperature across three groups of dwarfs.

In [12]:
# Perform ANOVA test and print the result table
anova_temp <- aov(Temperature..K.~Star.type, data = dwarfs_temp)%>%
              tidy()
print(anova_temp)

# A tibble: 2 × 6
  term         df       sumsq      meansq statistic   p.value
  <chr>     <dbl>       <dbl>       <dbl>     <dbl>     <dbl>
1 Star.type     2 3106600865. 1553300433.      188.  2.74e-37
2 Residuals   117  965696814.    8253819.       NA  NA       


In [24]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_temp <- anova_temp %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_temp <- anova_temp %>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_temp)
print(anova_pval_temp)

[1] 188.1917
[1] 2.738892e-37


The F-value obtained is 188.91, and the p-value is 2.74e-37. In this case, our F-value is significantly greater than 1. Therefore, the variation between group means is clearly larger than the variation within groups, indicating that at least one of the mean value is different across three groups. The p-value also states the result. Since p-value is significantly smaller than the critical value (0.05), it falls in reject region. Therefore, by looking at the ANOVA test for temperature across three groups, we say at least one of the mean value is different across three groups and the temperature of three dwarf groups are not all same.

### ANOVA for Luminosity Across all Three Dwarf Types

We perform ANOVA test for luminosity in the following section. First, we select the target columns.

In [14]:
# filter out the target column Luminosity to get the tidy data set
dwarfs_lumi <- dwarfs %>%
               select(Luminosity.L.Lo., Star.type)
head(dwarfs_lumi)

,Luminosity.L.Lo.,Star.type
,<dbl>,<fct>
1,0.002400,0
2,0.000500,0
3,0.000300,0
4,0.000200,0
5,0.000138,0
6,0.000650,0


In the following two blocks of code, we perform ANOVA test on luminosity across three groups of dwarfs.

In [18]:
# Perform ANOVA test and print the result table
anova_lumi <- aov(Luminosity.L.Lo.~Star.type, data = dwarfs_lumi)%>%
              tidy()
print(anova_lumi)

# A tibble: 2 × 6
  term         df    sumsq    meansq statistic  p.value
  <chr>     <dbl>    <dbl>     <dbl>     <dbl>    <dbl>
1 Star.type     2 0.000454 0.000227       5.09  0.00761
2 Residuals   117 0.00522  0.0000447     NA    NA      


In [19]:
# Filter out the F-value from the table above
# Filter out the p-value from the table above
f_stat_lumi <- anova_lumi %>%
          filter(term == "Star.type") %>%
          select(statistic) %>%
          as.numeric()
anova_pval_lumi <- anova_lumi%>%
              filter(term == "Star.type") %>%
              select(p.value) %>%
              as.numeric()
print(f_stat_lumi)
print(anova_pval_lumi)

[1] 5.086809
[1] 0.007614593


The F-value obtained is 5.09, and the p-value is 0.0076. Although we get a much smaller F-value than the F-value in temperature test, it still clearly larger than 1. The p-value is also smaller than the critical value (0.05), and falls in reject region. Although we might say the difference between luminosity across groups is not as significantly as the difference between temperature, we are still confident to say at least one mean value is different from others, and the luminosity of three dwarf groups are not all same.

# Citation

Zach. (2021, May 11). What does a high F value mean in ANOVA? Statology. Retrieved April 4, 2022, from https://www.statology.org/what-does-a-high-f-value-mean

# Might not be useful

Before actually starting the ANOVA test, we will start with a general analysis of variance. In this section, we calculate the ratio between the overall variance across three groups and the average of each within-group variances. If there is a significant difference between the two numbers, it should give us a rough idea that the average temperature of the three dwarf planets might be different.

In [22]:
# Calculate the within-group variance of temperature for each star type
variance_temp <- 
    dwarfs_temp %>%
    group_by(Star.type)%>%
    summarise(variance = var(Temperature..K.))

variance_temp

Star.type,variance
<fct>,<dbl>
0,110407.54
1,72704.25
2,24578344.97


In [23]:
# Calculate the overall variance of temperature across groups
# Calculate the average of each within-group variance of temeprature
# Calculate the ratio between two above data
variance_overall_temp <- 
    dwarfs_temp %>% 
    summarise(overall_var = var(Temperature..K.)) %>% 
    pull(overall_var)

variance_group_avg_temp <- 
    variance_temp%>%
    summarise(group_var_avg = mean(variance)) %>%
    pull(group_var_avg)

variance_ratio_temp <- variance_overall_temp/variance_group_avg_temp

variance_ratio_temp

[1] 4.146079

In [25]:
variance_lumi <- 
    dwarfs_lumi %>%
    group_by(Star.type)%>%
    summarise(variance = var(Luminosity.L.Lo.))

variance_lumi

Star.type,variance
<fct>,<dbl>
0,7.883766e-07
1,5.373242e-05
2,7.943258e-05


In [26]:
variance_overall_lumi <- 
    dwarfs_lumi %>% 
    summarise(overall_var = var(Luminosity.L.Lo.)) %>% 
    pull(overall_var)

variance_group_avg_lumi <- 
    variance_lumi %>%
    summarise(group_var_avg = mean(variance)) %>%
    pull(group_var_avg)

variance_ratio_lumi <- variance_overall_lumi/variance_group_avg_lumi

variance_ratio_lumi

[1] 1.068686